In [1]:
import argparse
import sys
import ast
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

sys.path.append("/media/alexander/SSD990PRO/sk_harmonization/FastSAM/")

from fastsam import FastSAM, FastSAMPrompt 
from utils.tools import convert_box_xywh_to_xyxy

In [2]:
model_path = "/media/alexander/SSD990PRO/sk_harmonization/FastSAM/weights/FastSAM-x.pt"
model = FastSAM(model_path)

/media/alexander/SSD990PRO/sk_harmonization/FastSAM/ultralytics/nn/tasks.py:518: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), f

In [3]:
class Args: 
    def __init__(self, ):
        self.imgsz = 1024  # image size
        self.iou = 0.9  # iou threshold for filtering the annotations
        self.conf = 0.4  # object confidence threshold
        self.text_prompt = None
        self.output = "./FastSAM/output/"
        self.randomcolor = False
        self.point_prompt = "[[0,0]]"  # [[x1,y1],[x2,y2]]
        self.point_label = "[0]"  #[ 1,0] 0:background, 1:foreground
        self.box_prompt = "[[0,0,0,0]]"  # [[x,y,w,h],[x2,y2,w2,h2]] support multiple boxes
        self.better_quality = True   # "better quality using morphologyEx",
        self.device = "cuda:0"
        self.retina = True  # draw high-resolution segmentation masks
        self.withContours = False  # draw edges


def main(args, img_path):
    args.point_prompt = ast.literal_eval(args.point_prompt)
    args.box_prompt = convert_box_xywh_to_xyxy(ast.literal_eval(args.box_prompt))
    args.point_label = ast.literal_eval(args.point_label)
    input = Image.open(img_path)
    input = input.convert("RGB")
    everything_results = model(
        input,
        device=args.device,
        retina_masks=args.retina,
        imgsz=args.imgsz,
        conf=args.conf,
        iou=args.iou    
        )
    bboxes = None
    points = None
    point_label = None
    prompt_process = FastSAMPrompt(input, everything_results, device=args.device)
    if args.box_prompt[0][2] != 0 and args.box_prompt[0][3] != 0:
            ann = prompt_process.box_prompt(bboxes=args.box_prompt)
            bboxes = args.box_prompt
    elif args.text_prompt != None:
        ann = prompt_process.text_prompt(text=args.text_prompt)
    elif args.point_prompt[0] != [0, 0]:
        ann = prompt_process.point_prompt(
            points=args.point_prompt, pointlabel=args.point_label
        )
        points = args.point_prompt
        point_label = args.point_label
    else:
        ann = prompt_process.everything_prompt()
    prompt_process.plot(
        annotations=ann,
        output_path=args.output+img_path.split("/")[-1],
        bboxes = bboxes,
        points = points,
        point_label = point_label,
        withContours=args.withContours,
        better_quality=args.better_quality,
    )
    return args.output+img_path.split("/")[-1]

In [1922]:
n=22

In [ ]:
# im_path = "./FastSAM/images/dogs.jpg"
im_path = "/media/alexander/DATA/dima_vinichenko/data/sd_15/DDIM/" + str(n) + ".png"
# im_path = "./data/imgs/pink_suit.png"

args = Args()
args.text_prompt = ""
# args.text_prompt = "a pink suit"
args.iou = 0.9
args.conf = 0.4
args.withContours = True
# args.point_label = "[1]"
# args.point_prompt = "[[512, 512]]"
out_path = main(args, im_path)
Image.open(out_path)

In [ ]:
everything_results = model(
        Image.open(im_path).convert('RGB'),
        device=args.device,
        retina_masks=args.retina,
        imgsz=args.imgsz,
        conf=args.conf,
        iou=args.iou
     )


mask_array_1 = everything_results[0].masks.data.cpu().numpy()[1, : ,:] 
plt.imshow(mask_array_1)

In [1886]:
# mask_array_1[mask_array_1 > 1] = 1
# plt.imshow(mask_array_1)

In [ ]:
# im_path = "./FastSAM/images/dogs.jpg"
im_path = "/media/alexander/DATA/dima_vinichenko/data/sd_15/DDPM_original/" + str(n) + ".png"
# im_path = "./data/imgs/pink_suit.png"

args = Args()
args.text_prompt = ""
# args.text_prompt = "a pink suit"
args.iou = 0.9
args.conf = 0.4
args.withContours = True
# args.point_label = "[1]"
# args.point_prompt = "[[512, 512]]"
out_path = main(args, im_path)
Image.open(out_path)

In [ ]:
everything_results = model(
        Image.open(im_path).convert('RGB'),
        device=args.device,
        retina_masks=args.retina,
        imgsz=args.imgsz,
        conf=args.conf,
        iou=args.iou
        )


mask_array_2 = everything_results[0].masks.data.cpu().numpy()[3, : ,:] 
plt.imshow(mask_array_2)

In [ ]:
# im_path = "./FastSAM/images/dogs.jpg"
im_path = "/media/alexander/DATA/dima_vinichenko/data/sd_15/DPM/" + str(n) + ".png"
# im_path = "./data/imgs/pink_suit.png"

args = Args()
args.text_prompt = ""
# args.text_prompt = "a pink suit"
args.iou = 0.9
args.conf = 0.4
args.withContours = True
# args.point_label = "[1]"
# args.point_prompt = "[[512, 512]]"
out_path = main(args, im_path)
Image.open(out_path)

In [ ]:
everything_results = model(
        Image.open(im_path).convert('RGB'),
        device=args.device,
        retina_masks=args.retina,
        imgsz=args.imgsz,
        conf=args.conf,
        iou=args.iou
        )

mask_array_3 = everything_results[0].masks.data.cpu().numpy()[1, : ,:] 
plt.imshow(mask_array_3)

In [1955]:
# mask_array_3[mask_array_3 > 1] = 1
# plt.imshow(mask_array_3)

In [1956]:
# mask_array_3[mask_array_3 == 2] = 1

In [ ]:
# im_path = "./FastSAM/images/dogs.jpg"
im_path = "/media/alexander/DATA/dima_vinichenko/data/sd_15/DPM_AYS/" + str(n) + ".png"
# im_path = "./data/imgs/pink_suit.png"

args = Args()
args.text_prompt = ""
# args.text_prompt = "a pink suit"
args.iou = 0.9
args.conf = 0.4
args.withContours = True
# args.point_label = "[1]"
# args.point_prompt = "[[512, 512]]"
out_path = main(args, im_path)
Image.open(out_path)

In [ ]:
everything_results = model(
        Image.open(im_path).convert('RGB'),
        device=args.device,
        retina_masks=args.retina,
        imgsz=args.imgsz,
        conf=args.conf,
        iou=args.iou
        )

mask_array_4 = everything_results[0].masks.data.cpu().numpy()[3, : ,:]
plt.imshow(mask_array_4)

In [1963]:
mask_array = mask_array_1.astype('bool') + mask_array_2.astype('bool') + mask_array_3.astype('bool') + mask_array_4.astype('bool')
mask_array = mask_array.astype('int')

In [1964]:
mask_array_back = 1 - mask_array

In [1965]:


image_tmp = np.array(Image.open(im_path))


In [1966]:
for i in range(3):
    image_tmp[:,:,i] = np.multiply(image_tmp[:,:,i], mask_array_back)

In [ ]:
img = Image.fromarray(image_tmp, 'RGB') 
img

In [1968]:
path = "/media/alexander/DATA/dima_vinichenko/data/sd_15/mask/"
np.save(path  + str(n), mask_array_back)
